# nio [JDK10都发布了，nio你了解多少？](https://mp.weixin.qq.com/s?__biz=MzI4Njg5MDA5NA==&mid=2247484235&idx=1&sn=4c3b6d13335245d4de1864672ea96256&chksm=ebd7424adca0cb5cb26eb51bca6542ab816388cf245d071b74891dd3f598ccd825f8611ca20c&scene=21###wechat_redirect)

## buffer代码演示
首先展示一下是如何创建缓冲区的，核心变量的值是怎么变化的

In [3]:
import java.nio.ByteBuffer;

In [39]:
public static void main() {

    // 创建一个缓冲区
    ByteBuffer byteBuffer = ByteBuffer.allocate(1024);

    // 看一下初始时4个核心变量的值
    System.out.println("初始时-->limit--->"+byteBuffer.limit());
    System.out.println("初始时-->position--->"+byteBuffer.position());
    System.out.println("初始时-->capacity--->"+byteBuffer.capacity());
    System.out.println("初始时-->mark--->" + byteBuffer.mark());

    System.out.println("--------------------------------------");

    // 添加一些数据到缓冲区中
    String s = "anlzou";
    byteBuffer.put(s.getBytes());

    // 看一下初始时4个核心变量的值
    System.out.println("put完之后-->limit--->"+byteBuffer.limit());
    System.out.println("put完之后-->position--->"+byteBuffer.position());
    System.out.println("put完之后-->capacity--->"+byteBuffer.capacity());
    System.out.println("put完之后-->mark--->" + byteBuffer.mark());
    
    System.out.println("--------------------------------------");
    // 切换成读模式
    byteBuffer.flip();
    // 看一下初始时4个核心变量的值
    System.out.println("flip完之后-->limit--->"+byteBuffer.limit());
    System.out.println("flip完之后-->position--->"+byteBuffer.position());
    System.out.println("flip完之后-->capacity--->"+byteBuffer.capacity());
    System.out.println("flip完之后-->mark--->" + byteBuffer.mark());
    
    System.out.println("--------------------------------------");
     // 创建一个limit()大小的字节数组(因为就只有limit这么多个数据可读)
    byte[] bytes = new byte[byteBuffer.limit()];

    // 将读取的数据装进我们的字节数组中
    byteBuffer.get(bytes);

    // 输出数据
    System.out.println(new String(bytes, 0, bytes.length));
}

In [40]:
main();

初始时-->limit--->1024
初始时-->position--->0
初始时-->capacity--->1024
初始时-->mark--->java.nio.HeapByteBuffer[pos=0 lim=1024 cap=1024]
--------------------------------------
put完之后-->limit--->1024
put完之后-->position--->6
put完之后-->capacity--->1024
put完之后-->mark--->java.nio.HeapByteBuffer[pos=6 lim=1024 cap=1024]
--------------------------------------
flip完之后-->limit--->6
flip完之后-->position--->0
flip完之后-->capacity--->1024
flip完之后-->mark--->java.nio.HeapByteBuffer[pos=0 lim=6 cap=1024]
--------------------------------------
anlzou


现在我想要从缓存区拿数据，怎么拿呀？？NIO给了我们一个`flip()`方法。这个方法可以改动position和limit的位置！

还是上面的代码，我们`flip()`一下后，再看看4个核心属性的值发生了变化：

很明显的是：

- limit变成了position的位置了
- 而position变成了0

看到这里的同学可能就会想到了：当调用完`filp()`时：limit是限制读到哪里，而position是从哪里读

一般我们称`filp(`)为“切换成读模式”
- 每当要从缓存区的时候读取数据时，就调用`filp()`“切换成读模式”。

In [48]:
public static void test(){
    // 创建一个缓冲区
    ByteBuffer byteBuffer = ByteBuffer.allocate(1024);

    // 添加一些数据到缓冲区中
    String s = "anlzou";
    byteBuffer.put(s.getBytes());
    
    
    /**
    * 切换成读模式
    * 使用该模式才能从缓冲区中读取数据
    */
    byteBuffer.flip();

    // 创建一个limit()大小的字节数组(因为就只有limit这么多个数据可读)
    byte[] bytes = new byte[byteBuffer.limit()];

    // 将读取的数据装进我们的字节数组中
    byteBuffer.get(bytes);

    // 输出数据
    System.out.println(new String(bytes, 0, bytes.length));
    
    
    /*-----------------清空缓冲区实验----------------*/
    // 清空缓冲区
    byteBuffer.clear();
    
    byte[] bytes_ = new byte[byteBuffer.limit()];
    
    // 将读取的数据装进我们的字节数组中
    byteBuffer.get(bytes_);
    
    // 输出数据
    System.out.println(new String(bytes_, 0, bytes_.length));
    
    System.out.println("--------------------------------------");
    // 看一下初始时4个核心变量的值
    System.out.println("clear完之后-->limit--->"+byteBuffer.limit());
    System.out.println("clear完之后-->position--->"+byteBuffer.position());
    System.out.println("clear完之后-->capacity--->"+byteBuffer.capacity());
    System.out.println("clear完之后-->mark--->" + byteBuffer.mark());
}

In [49]:
test();

anlzou
anlzou                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

读完我们还想写数据到缓冲区，那就使用`clear()`函数，这个函数会“清空”缓冲区：

- **数据没有真正被清空**，只是被**遗忘**掉了